In [1]:
# Uncomment and run this cell if you don't have DEAP or matplotlib installed (or haven't ran requirements.txt yet).
# !pip install deap matplotlib matplotlib

In [2]:
# Import necessary libraries
import random

from deap import base, creator, tools

import data_store.street_fighter.letters as letters

In [3]:
# Set letters to the desired value
LETTERS = letters.SIMPLE_LETTERS

In [4]:
# Define fitness function
def calculate_distance(layout, frequencies):
    positions = {letter: idx for idx, letter in enumerate(layout)}
    total_distance = 0
    for (letter1, freq1) in frequencies.items():
        for (letter2, freq2) in frequencies.items():
            if letter1 != letter2:
                dist = abs(positions[letter1] - positions[letter2])  # Linear distance
                total_distance += freq1 * freq2 * dist
    return total_distance,


In [5]:
# Create the Optimization Problem
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)


In [6]:
# Define a Custom Crossover Function
def cxOnePoint(ind1, ind2):
    if len(ind1) != len(ind2):
        raise ValueError("Individuals must have the same length.")
    
    size = len(ind1)
    cxpoint = random.randint(1, size - 1)  # Ensure cxpoint is within range

    ind1[cxpoint:], ind2[cxpoint:] = ind2[cxpoint:], ind1[cxpoint:]
    return ind1, ind2


In [7]:
# Define a Custom Mutation Function
def mutShuffleIndexes(individual, indpb):
    """Shuffle mutation for list-based individuals."""
    size = len(individual)
    for i in range(size):
        if random.random() < indpb:
            j = random.randint(0, size - 1)
            individual[i], individual[j] = individual[j], individual[i]
    return individual,

In [8]:
# Initialize Individuals and Population
def init_individual():
    # Initialize an individual with a random permutation of the letters
    layout = LETTERS[:]
    random.shuffle(layout)
    return creator.Individual(layout)

toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, lambda: random.sample(LETTERS, len(LETTERS)))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [9]:
# Define the Genetic Operators
toolbox.register("evaluate", calculate_distance, frequencies={
    "→": 20, "←": 15, "↑": 10, "↓": 10, "LP": 10, "MP": 15, "HP": 10, "LK": 5, "MK": 5, "HK": 5
})
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

In [10]:
# Evaluate Invalid Fitness
def evaluate_population(population, toolbox):
    for ind in population:
        if not ind.fitness.valid or len(ind.fitness.values) == 0:
            ind.fitness.values = toolbox.evaluate(ind)
            if not isinstance(ind.fitness.values, tuple) or len(ind.fitness.values) == 0:
                print(f"Warning: Fitness values for individual {ind} are invalid or empty")
    return population


In [11]:
# Debug Fitness Values
def debug_fitness(population):
    for ind in population:
        if not isinstance(ind.fitness.values, tuple):
            print(f"Warning: Fitness values for individual {ind} are not a tuple")
        if len(ind.fitness.values) == 0:
            print(f"Warning: Fitness values for individual {ind} are empty")
        else:
            print(f"Individual: {ind}, Fitness: {ind.fitness.values}")


In [15]:
def main():
    # Set DEAP parameters
    population = toolbox.population(n=100)
    NGEN = 50
    CXPB, MUTPB = 0.5, 0.2
    
    print("Starting the optimization")
    for gen in range(NGEN):
        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                try:
                    toolbox.mate(child1, child2)
                except IndexError as e:
                    print(f"IndexError during mating: {e}")
                    print(f"Child1: {child1}, Child2: {child2}")
                    continue  # Skip this crossover operation
                del child1.fitness.values
                del child2.fitness.values

        # Apply mutation
        for mutant in offspring:
            if random.random() < MUTPB:
                try:
                    toolbox.mutate(mutant)
                except IndexError as e:
                    print(f"IndexError during mutation: {e}")
                    print(f"Mutant: {mutant}")
                    continue  # Skip this mutation operation
                del mutant.fitness.values

        # Evaluate the fitness of individuals with invalid fitness
        population = evaluate_population(population, toolbox)
        debug_fitness(population)

        # Update the population
        population[:] = toolbox.select(population + offspring, len(population))

        # Print statistics
        try:
            fits = [ind.fitness.values[0] for ind in population]
        except IndexError as e:
            print(f"IndexError while accessing fitness values: {e}")
            print(f"Population fitness values: {[ind.fitness.values for ind in population]}")
            continue  # Skip this generation

        length = len(population)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print(f"Generation {gen}: Min {min(fits)} | Max {max(fits)} | Avg {mean} | Std {std}")

    return population

In [16]:
main()

Starting the optimization
Individual: ['↓', 'MK', 'LP', '←', 'HP', '→', 'HK', '↑', 'MP', 'LK'], Fitness: (33100.0,)
Individual: ['MK', 'LK', 'HK', '→', 'LP', '←', 'MP', 'HP', '↓', '↑'], Fitness: (31100.0,)
Individual: ['LK', '←', '↑', '→', 'LP', 'MK', 'HP', 'HK', '↓', 'MP'], Fitness: (36200.0,)
Individual: ['→', 'HK', 'HP', '↑', 'LP', '↓', 'LK', 'MP', '←', 'MK'], Fitness: (37700.0,)
Individual: ['MK', 'MP', 'HK', 'HP', '↓', 'LP', '←', '↑', 'LK', '→'], Fitness: (36500.0,)
Individual: ['→', '↑', 'MP', 'MK', 'LP', 'HK', '←', 'HP', '↓', 'LK'], Fitness: (37100.0,)
Individual: ['HK', 'MK', '↑', '←', '↓', 'MP', 'HP', '→', 'LP', 'LK'], Fitness: (30600.0,)
Individual: ['LK', '↓', '→', 'LP', 'HK', '←', 'MK', 'MP', 'HP', '↑'], Fitness: (34900.0,)
Individual: ['MK', 'HK', '↓', 'LP', 'LK', '←', 'MP', '↑', '→', 'HP'], Fitness: (32800.0,)
Individual: ['→', '←', 'HP', '↑', 'MP', 'LK', '↓', 'MK', 'HK', 'LP'], Fitness: (36600.0,)
Individual: ['MK', '→', '←', '↓', 'HP', 'LK', 'MP', 'HK', '↑', 'LP'], Fitn

KeyError: '↑'